# SV (Space Vehicle) Models

Load `gnss_lib_py` into the Python workspace

In [ ]:
import gnss_lib_py as glp

## Adding SV States with Precise Ephemerides (SP3 & CLK)

This tutorial explains how to calculate satellite states using precise SP3 and CLK files.

The data required to calculate with precise ephemerides uses .sp3 and .clk files, which can be downloaded from [CDDIS](https://cddis.nasa.gov/Data_and_Derived_Products/GNSS/gnss_mgex.html) or [CORS](https://geodesy.noaa.gov/UFCORS/).

The .sp3 files provide post-processed, accurate, and precise information regarding 3-D satellite position in the Earth-Centered Earth-Fixed (ECEF) frame at intervals of 15mins each.  

Similarly, the .clk files provide post-processed, accurate and precise information on satellite clock errors at intervals of 30secs each.  

These .sp3 and .clk files are available for any GNSS constellation, and hence, provide a common processing platform for applications that involve multi-GNSS satellite signals (without requiring to parse the broadcast ephemeris from each constellation separately one at a time). Also, unlike broadcast ephemeris that can suffer from signal-in-space anomalies, the .sp3 and .clk files are guaranteed to provide accurate satellite information. However, note that, these files are only available in a post-processed manner, and not in real-time

We show how to analyze this precise ephemerides functionality for the Android derived dataset in the following cells, 

1. Load the derived data from AndroidDerived and remove the rows in NavData class that refer to satellite information (3-D satellite position, 3-D satellite velocity, clock bias and clock drift),

In [ ]:
import numpy as np

# load Android Google Challenge data
glp.make_dir("../data")
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/google_decimeter_2021/Pixel4_derived_clkdiscnt.csv --quiet -nc -O "../data/Pixel4_derived_clkdiscnt.csv"
derived_data = glp.AndroidDerived2021("../data/Pixel4_derived_clkdiscnt.csv", remove_timing_outliers=False)
# Define the keys relevant for satellite information, and remove the data within these fields
SV_KEYS = ['x_sv_m', 'y_sv_m', 'z_sv_m', \
           'vx_sv_mps','vy_sv_mps','vz_sv_mps', \
           'b_sv_m', 'b_dot_sv_mps']
derived_data.remove(rows=SV_KEYS,inplace=True)

2. Specify the paths to the .sp3 and .clk files using the `file_path` variable. If files are not specified, they will be automatically downloaded using the ephemeris downloader.

In [ ]:
# download .sp3 data file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/sp3/COD0MGXFIN_20211180000_01D_05M_ORB.SP3 --quiet -nc -O "../data/COD0MGXFIN_20211180000_01D_05M_ORB.SP3"
# Specify .sp3 file path to extract precise ephemerides
sp3_path = "../data/COD0MGXFIN_20211180000_01D_05M_ORB.SP3"

# download .clk data file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/clk/COD0MGXFIN_20211180000_01D_30S_CLK.CLK --quiet -nc -O "../data/COD0MGXFIN_20211180000_01D_30S_CLK.CLK"
# Specify .clk file path to extract precise ephemerides
clk_path = "../data/COD0MGXFIN_20211180000_01D_30S_CLK.CLK"

3. Populate the columns of SV_KEYS with information extracted via precise ephemerides

In [ ]:
# Update derived_data class with satellite information computed via precise ephemerides
derived_multi_gnss = glp.add_sv_states(derived_data, source="precise", file_paths=[sp3_path, clk_path],
                                       verbose = False)

Check that all the desired fields related to satellite information have useful information and the norm of computed satellite position matches the altitude of GNSS constellations

In [ ]:

sat_alt = np.linalg.norm(derived_multi_gnss[["x_sv_m","y_sv_m","z_sv_m"],9:12],axis=0)
print('Distance of two satellites from the center of the Earth (expected around 26000000 m)')
print("Three GPS SVs calculated to be at:", sat_alt,"\n")

print("Small section of calculated positions:")
print(derived_multi_gnss.copy(cols=[2,3,4,5],rows=["gnss_id"]+SV_KEYS))

## Adding SV states to `NavData` with received measurements using broadcast ephemeris

This tutorial explains how to estimate SV states, both as estimation for
a single time instance and measurements and as a wrapper for an entire
set of received measurements.

For this tutorial, we will work with the `AndroidDerived2022` dataset.
This serves the dual purpose of showing how each functionality works and
allowing us to compare the SV states estimated in `sv_models.py` to that
estimated by Google. The latter verifies state computation from our method.

Load the test dataset for the Android Derived 2022 dataset

In [ ]:
import numpy as np
# download Android data file
glp.make_dir("../data")
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/google_decimeter_2022/device_gnss.csv --quiet -nc -O "../data/device_gnss.csv"
# load Android Google Challenge data
derived_data = glp.AndroidDerived2022("../data/device_gnss.csv")

Create a copy of `derived_data` for comparison later on and remove
Google computed SV states from `derived_data`. 

These states will be added to `derived_data` using functions from `gnss_lib_py`.

Currently we only support GPS satellites for SV state estimation with
broadcast ephemeris parameters. Support for other constellations is 
coming soon.

Now, estimate SV states and add them to `derived_data` using `gnss_lib_py`.

In [ ]:
derived_gps = derived_data.where("gnss_id", "gps")
derived_l1 = derived_gps.where("signal_type", "l1")
derived_reference = derived_l1.copy()
sv_state_rows = ['x_sv_m', 'y_sv_m', 'z_sv_m', 'vx_sv_mps', 'vy_sv_mps', 'vz_sv_mps', 'b_sv_m']
derived_l1.remove(rows=sv_state_rows, inplace=True)
derived_sv_states = glp.add_sv_states_rinex(derived_l1)

For sanity checking, we compare our estimated SV states to those computed
by Google and print out the maximum absolute errors

In [ ]:
print('Estimating differences between estimated SV states and Google reference')
for row in sv_state_rows:
    mean_diff = np.mean(derived_reference[row] - derived_sv_states[row])
    max_diff = np.max(np.abs(derived_reference[row] - derived_sv_states[row]))
    print(f"For row {row}, the max error is {max_diff:.4f}, the mean error is {mean_diff:.4f}")

## Add SV states for visible satellites given a series of times and positions

Given a series of times and a trajectory, find visible satellites 
(assuming open sky conditions) for those positions and times

In [ ]:
# Construct a linear trajectory starting from the Durand building and
# moving North at 10 m/s for 10 steps
# This trajectory will not have any major differences in calculated
# satellite positions but is to demonstrate the functionality

from datetime import datetime, timezone
# Send time at which SV states are needed in GPS millis
start_time = datetime(year=2021,
                       month=4,
                       day=29,
                       hour=22,
                       minute=30,
                       second=0)
start_time = start_time.replace(tzinfo=timezone.utc)
start_gps_millis = glp.datetime_to_gps_millis(start_time)

# Create sequence of times
gps_millis_traj = start_gps_millis + 1000.*np.arange(10)

# Define receiver position in ECEF
rx_LLA_durand = np.reshape([37.427112, -122.1764146, 16], [3, 1])
rx_ecef_durand = np.reshape(glp.geodetic_to_ecef(rx_LLA_durand), [3, 1])

# Create sequence of moving receiver (using approximation of long to meters)
rx_LLA_traj = rx_LLA_durand + np.vstack((np.zeros(10), 
                                         0.0001*10.*np.arange(10), 
                                         np.zeros(10)))

# Convert trajectory to ECEF 
rx_ecef_traj = glp.geodetic_to_ecef(rx_LLA_traj)

# Create state estimate with given trajectory
state_traj = glp.NavData()
state_traj['gps_millis'] = gps_millis_traj
state_traj['x_rx_m'] = rx_ecef_traj[0,:]
state_traj['y_rx_m'] = rx_ecef_traj[1,:]
state_traj['z_rx_m'] = rx_ecef_traj[2,:]

# Define all GPS satellites, so that all broadcast ephemeris parameters
# are downloaded
gps_all_sats = [f"G{sv:02}" for sv in range(1, 33)]

# Download ephemeris files for given time
ephem_all_sats = glp.get_time_cropped_rinex(start_gps_millis, gps_all_sats, ephemeris_directory="ephemeris")

With the setup done, we now pass these parameters to `add_visible_svs_for_trajectory`
to add visible satellites corresponding to the times and positions of the
trajectory.

In [ ]:
sv_posvel_traj = glp.add_visible_svs_for_trajectory(state_traj,
                                                    ephemeris_path="ephemeris")


For reference, we demonstrate the changing satellite positions for SV ID
30

In [ ]:
sv_posvel_traj_sv25 = sv_posvel_traj.where("sv_id", 25)

print('GPS milliseconds with first time subtracted\n', 
        sv_posvel_traj_sv25['gps_millis'] - start_gps_millis)

print('Changing x ECEF SV positions\n',
        sv_posvel_traj_sv25['x_sv_m'] - sv_posvel_traj_sv25['x_sv_m', 0])

print('Consecutive change in x ECEF positions\n', 
        sv_posvel_traj_sv25['x_sv_m', 1:] - sv_posvel_traj_sv25['x_sv_m', :-1])

print('Velocity along x ECEF for reference\n',
        sv_posvel_traj_sv25['vx_sv_mps'])

## Finding PRNs and states for visible SVs for a given position and time

Consider the problem of states of SVs that would be visible from Durand
building (considering open sky with an elevation mask of 5&deg;) on the 
Stanford campus on 30th April, 2021.

In this section, we show how to estimate the visible satellites and then
compute their states

In [ ]:
# Uses start_time, start_gps_millis, rx_ecef_durand, and ephem from previous
# section



# Use input time, Rx position and all broadcast ephemeris parameters to
# find ephemeris parameters for visible satellites

ephem_viz = glp.find_visible_ephem(start_gps_millis, rx_ecef_durand, ephem_all_sats)

print("SV IDs for visible satellites are ", ephem_viz['sv_id'])


## Finding SV states at given time and for specific PRNs

Using the ephemeris parameters, we can find SV states for those specific
PRNs.

`gnss_lib_py` offers two options to find SV states:
1. Estimating SV states at precisely the given time
2. Estimating SV states for the given reception time. This subtracts the
   time of travel for the signal (based on the receiver position) and
   computes the SV states at that approximate transmission time.
   This method requires an estimate of the receiver's position.

The time taken for signals to reach the Earth from satellites is roughly
70 ms and the difference between SV positions is roughly 200 m.

In this section, we show both methods of estimating SV states.

In [ ]:
# Using find_sv_states and a filtered ephem
# Option 1: Estimate SV states for the given transmission time (does not 
# account for any signal travel time)
sv_states_tx = glp.find_sv_states(start_gps_millis, ephem_viz)

# Option 2: Estimate SV states for given reception time (factors and removes
# approximately the time taken by the signal to reach the receiver)
# This method requires an estimate of the receiver's position and also
# gives difference between positions and the estimated true range
sv_states_rx, del_pos, true_range = glp.find_sv_location(start_gps_millis, rx_ecef_durand, ephem_viz)

print('Difference between x positions estimated for Tx and Rx times \n', 
      sv_states_tx['x_sv_m'] - sv_states_rx['x_sv_m'])
print('Difference between x velocities estimated for Tx and Rx times\n', 
      sv_states_tx['vx_sv_mps'] - sv_states_rx['vx_sv_mps'])


## Simulating SV positions given elevation and azimuth

When working in a local frame of reference, it can be faster to simulate
satellite positions locally, based on elevation and azimuth angles.

In this section, we demonstrate how to achieve this by giving an `np.ndarray`
containing elevation and azimuth angles to get an `np.ndarray` 

In [ ]:
# Use svs_from_el_az
elevation = np.array([30., 45., 60., 30.])
azimuth = np.linspace(0, 360., 4, endpoint=False)
el_az = np.vstack((elevation, azimuth))
local_svs = glp.svs_from_el_az(el_az)

print('Given elevation angles are ', elevation)
print('Given azimuth angles are ', azimuth)
print('Local coordinates for generated satellites are \n', local_svs)
